In [17]:
!pip install Unidecode
!pip install num2words

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13775 sha256=7346093d1a409474c893abb709c6b0c7896ff87924761db0e7daccb095e4a56a
  Stored in directory: c:\users\pablo\appdata\local\pip\cache\wheels\1a\bf\a1\4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt


In [18]:
import pandas as pd
import re
from unidecode import unidecode
from num2words import num2words

In [ ]:
df=pd.read_csv('fake_news_spanish.csv', encoding='utf-8', delimiter=';')
df

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018
...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022


In [ ]:
df['Titulo']=df['Titulo'].fillna('')
df['d']=df['Titulo']+df['Descripcion']
df


,ID,Label,Titulo,Descripcion,Fecha,d
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,'The Guardian' va con Sánchez: 'Europa necesit...
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,El 'Ahora o nunca' de Joan Fuster sobre el est...
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild..."
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,Puigdemont: 'No sería ninguna tragedia una rep...
...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,El Defensor del Pueblo reclama a la Comunidad ...
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,El EQUO plantea ceder la presidencia de la Com...
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,Alberto Garzón: 'Que los Borbones son unos lad...
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,Vox exige entrar en el Gobierno de Castilla y ...


In [ ]:
def normalizar(texto):
    # todo a minúsculas
    texto = texto.lower()

    # tildes y diacríticas
    texto = unidecode(texto)

    return texto


In [ ]:
df['d']=df['d'].apply(normalizar)
df


,ID,Label,Titulo,Descripcion,Fecha,d
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,'the guardian' va con sanchez: 'europa necesit...
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,revelan que el gobierno negocio la liberacion ...
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,el 'ahora o nunca' de joan fuster sobre el est...
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,"iglesias alienta a yolanda diaz, erc y eh bild..."
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,puigdemont: 'no seria ninguna tragedia una rep...
...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,el defensor del pueblo reclama a la comunidad ...
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,el equo plantea ceder la presidencia de la com...
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,alberto garzon: 'que los borbones son unos lad...
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,vox exige entrar en el gobierno de castilla y ...


In [ ]:
def tokenizar(texto):
    # dividir el texto en palabras
    palabras = re.findall(r'\b\w+\b', texto)
    return palabras

df['d']=df['d'].apply(tokenizar)
df
df['d']

0        [the, guardian, va, con, sanchez, europa, nece...
1        [revelan, que, el, gobierno, negocio, la, libe...
2        [el, ahora, o, nunca, de, joan, fuster, sobre,...
3        [iglesias, alienta, a, yolanda, diaz, erc, y, ...
4        [puigdemont, no, seria, ninguna, tragedia, una...
                               ...                        
57058    [el, defensor, del, pueblo, reclama, a, la, co...
57059    [el, equo, plantea, ceder, la, presidencia, de...
57060    [alberto, garzon, que, los, borbones, son, uno...
57061    [vox, exige, entrar, en, el, gobierno, de, cas...
57062    [unas, 300, personas, protestan, contra, la, v...
Name: d, Length: 57063, dtype: object

In [24]:
def num_to_word(line):
    new_line=[]
    x=r'\d'
    for word in line:
        if re.search(x,word):
            new_line.append(num2words(word, to = 'ordinal', lang = 'es'))
        else:
            new_line.append(word)
    return new_line
    
df['d']=df['d'].apply(num_to_word)
df

InvalidOperation: [<class 'decimal.ConversionSyntax'>]